In [ ]:
#Adan tariq
#fa23-bbd-011

import pandas as pd
import numpy as np
import gradio as gr

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import joblib

In [3]:
# Load dataset
df = pd.read_excel(r"C:\Users\dell\Downloads\traffic-accidents-annual-.xlsx", engine="openpyxl")





In [4]:
df

,*- Islamabad Included.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Year,Total number of accidents,Fatal Accidents,Non-Fatal Accidents,Killed,Injured,Total number of vehicles involved
2,Pakistan,2008-09,9496,4145,5351,4907,11037,10322
3,Pakistan,2009-10,9747,4378,5369,5280,11173,10496
4,Pakistan,2010-11,9723,4280,5443,5271,11383,10822
...,...,...,...,...,...,...,...,...
59,Islamabad,2014-15,216,107,109,118,182,217
60,Islamabad,2015-16,244,120,124,140,209,244
61,Islamabad,2016-17,226,127,99,129,124,216
62,Islamabad,2017-18,259,157,102,167,162,259


In [5]:
df.columns = ["Country", "Year", "Total_Accidents", "Fatal_Accidents",
              "Non_Fatal_Accidents", "Killed", "Injured", "Vehicles_Involved"]

df = df.dropna()

In [6]:
df

,Country,Year,Total_Accidents,Fatal_Accidents,Non_Fatal_Accidents,Killed,Injured,Vehicles_Involved
2,Pakistan,2008-09,9496,4145,5351,4907,11037,10322
3,Pakistan,2009-10,9747,4378,5369,5280,11173,10496
4,Pakistan,2010-11,9723,4280,5443,5271,11383,10822
5,Pakistan,2011-12,9140,3966,5174,4758,10145,9986
6,Pakistan,2012-13 *,8988,3884,5104,4719,9710,9876
...,...,...,...,...,...,...,...,...
59,Islamabad,2014-15,216,107,109,118,182,217
60,Islamabad,2015-16,244,120,124,140,209,244
61,Islamabad,2016-17,226,127,99,129,124,216
62,Islamabad,2017-18,259,157,102,167,162,259


In [7]:
df["Risk_Level"] = df["Fatal_Accidents"].apply(
    lambda x: 1 if x > df["Fatal_Accidents"].mean() else 0
)

C:\Users\dell\AppData\Local\Temp\ipykernel_8036\141160622.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Risk_Level"] = df["Fatal_Accidents"].apply(


In [8]:
X = df[["Total_Accidents", "Non_Fatal_Accidents", "Injured", "Vehicles_Involved"]]
y = df["Risk_Level"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [10]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [12]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

In [13]:
def predict_risk(total_accidents, non_fatal, injured, vehicles):

    input_data = pd.DataFrame([[total_accidents, non_fatal, injured, vehicles]],
                              columns=["Total_Accidents", "Non_Fatal_Accidents",
                                       "Injured", "Vehicles_Involved"])

    input_scaled = scaler.transform(input_data)
    prediction = model.predict(input_scaled)

    if prediction[0] == 1:
        return "HIGH ACCIDENT RISK"
    else:
        return "LOW ACCIDENT RISK"

In [14]:
app = gr.Interface(
    fn=predict_risk,
    inputs=[
        gr.Number(label="Total Accidents"),
        gr.Number(label="Non-Fatal Accidents"),
        gr.Number(label="Injured"),
        gr.Number(label="Vehicles Involved")
    ],
    outputs="text",
    title="Road Accident Risk Predictor",
    description=f"Machine Learning Based Risk Prediction System\n\nModel Accuracy: {accuracy*100:.2f}%"
)

app.launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
